In [2]:
#Plot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Data Packages
import math
import pandas
import numpy as np
import pandas as pd
#Progress bar
from tqdm import tqdm

#Counter
from collections import Counter

#Operation
import operator

#Natural Language Processing Packages
import re
import nltk

## Download Resources
nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tag import PerceptronTagger
from nltk.data import find

## Machine Learning
import sklearn
import sklearn.metrics as metrics

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Litos\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Litos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Litos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Litos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Use vader to evaluated sentiment of reviews
def evalSentences(sentences, to_df=False, columns=[]):
    #Instantiate an instance to access SentimentIntensityAnalyzer class
    sid = SentimentIntensityAnalyzer()
    pdlist = []
    if to_df:
        for sentence in tqdm(sentences):
            ss = sid.polarity_scores(sentence)
            pdlist.append([sentence]+[ss['compound']])
        reviewDf = pandas.DataFrame(pdlist)
        reviewDf.columns = columns
        return reviewDf
    
    else:
        for sentence in tqdm(sentences):
            print(sentence)
            ss = sid.polarity_scores(sentence)
            for k in sorted(ss):
                print('{0}: {1}, '.format(k, ss[k]), end='')
            print()

# Load Data

In [5]:
cur_df = pd.read_csv('../../data/yelp/Data.csv', index_col=0,  encoding='latin-1')

In [6]:
reviews = cur_df['review_text'].values

In [7]:
reviewDF = evalSentences(reviews, to_df=True, columns=['reviewCol','vader'])

100%|█████████████████████████████████████████████████████████████████████████| 169961/169961 [05:04<00:00, 557.57it/s]


In [8]:
reviewDF.head()

,reviewCol,vader
0,I ordered the lemon mango slush and the lemon ...,0.9758
1,"Came here on a Sunday afternoon, it wasn't bus...",0.9763
2,{Grapefruit & Yakult Green Tea with Aloe Jelly...,0.9170
3,Saw this newly opened bubble tea shop and want...,0.9898
4,Happy Lemon has become my new favourite place ...,0.9922


In [97]:
cur_df['Binary_vader'] = (reviewDF['vader']> 0.05)*[1]

In [9]:
# #Save
# reviewDF.to_csv('../../data/yelp/reviewDF.csv')

# Find most sentiment words by Mutual Information

## Data preprocessing

In [15]:
# Note: You may want to use an NLTK tokenizer instead of a regular expression in the following
def dataFrameTransformation(cur_df, reviewDF, k=500):
    text = reviewDF['reviewCol'].values
    

#     text = text.replace('.',' ').replace('/t',' ').replace('\t',' ').replace('/',' ').replace('-',' ')
    text = np.char.replace(text, '.',' ')
    
    # Tokenize
#     text = nltk.word_tokenize(reviews)
    # Lowercase
    text = [w.lower() for w in text]
    # Remove Punctuation
    table = str.maketrans('', '', string.punctuation)
    text = [w.translate(table) for w in text]
    # Remove tokens that are not alphabetic
    text = [w for w in text if w.isalpha()]
    
    # Remove Stopwords
    # Get english stopwords
    en_stopwords = set(stopwords.words('english'))
    en_stopwords.remove('off')
    text = [w for w in text if w not in en_stopwords]
    
#     # Lemmatizing
#     lemmatizer = WordNetLemmatizer()
#     text = [lemmatizer.lemmatize(w) for w in text]

#     text = " " + " ".join(str(x) for x in text) + " "

#     text = text.replace('whitish', 'white')
#     text = text.replace('bisquity', ' biscuit ')
#     text = text.replace('carmel', ' caramel ')
#     text = text.replace('flower', ' floral ')
#     text = text.replace('piny', ' pine ')
#     text = text.replace('off white', 'offwhite')
#     text = text.replace('goden', 'gold')
#     text = text.replace('yello', 'yellow')
#     text = text.replace('reddish', ' red ') 
    
#     # Reset to token
#     text = nltk.word_tokenize(text)
#     table = str.maketrans('', '', string.punctuation)
#     text = [w.translate(table) for w in text]
#     text = [w for w in text if w.isalpha()]
# #     en_stopwords = set(stopwords.words('english'))
#     text = [w for w in text if w not in en_stopwords]
    
    # Top-k frequent terms
    counter = Counter()
    reviews = text
    for review in reviews:
            counter.update([word.lower() 
                            for word 
                            in re.findall(r'\w+', review) 
                            if word.lower() not in stop and len(word) > 2])
    topk = counter.most_common(k)        
    
    #Find out if a particular review has the word from topk list
    freqReview = []
    for i in range(len(reviews)):
        tempCounter = Counter([word.lower() for word in re.findall(r'\w+',reviews[i])])
        topkinReview = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
        freqReview.append(topkinReview)
        
        
    #Prepare freqReviewDf
    freqReviewDf = pandas.DataFrame(freqReview)
    dfName = []
    for c in topk:
        dfName.append(c[0])
    freqReviewDf.columns = dfName
    finalreviewDf = reviewDF.join(freqReviewDf)
    finaldf = hotelDf[['hotelName','ratingScore','groundTruth']].join(finalreviewDf)
    return topk, finaldf

In [20]:
# Note: You may want to use an NLTK tokenizer instead of a regular expression in the following
def dataFrameTransformation(hotelDf, reviewDF, k=500):
    reviews = reviewDF['reviewCol'].values
    
    stop = set(stopwords.words('english'))
    # Lowercase
    reviews = [w.lower() for w in reviews]
    
    # Top-k frequent terms
    counter = Counter()
    for review in tqdm(reviews):
            counter.update([word.lower() 
                            for word 
                            in re.findall(r'\w+', review) 
                            if word.lower() not in stop and len(word) > 2])
    topk = counter.most_common(k)        
    
    #Find out if a particular review has the word from topk list
    freqReview = []
    for i in tqdm(range(len(reviews))):
        tempCounter = Counter([word.lower() for word in re.findall(r'\w+',reviews[i])])
        topkinReview = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
        freqReview.append(topkinReview)
        
        
    #Prepare freqReviewDf
    freqReviewDf = pandas.DataFrame(freqReview)
    dfName = []
    for c in topk:
        dfName.append(c[0])
    freqReviewDf.columns = dfName
    finalreviewDf = reviewDF.join(freqReviewDf)
    finaldf = hotelDf.join(finalreviewDf)
    return topk, finaldf

topk, finaldf = dataFrameTransformation(cur_df, reviewDF, k=500)

# Compute Mutual Info

In [22]:
# get Top K mutual information terms from the dataframe
def getMI(topk, df, label_column='groundTruth'):
    miScore = []
    for word in topk:
        miScore.append([word[0]]+[metrics.mutual_info_score(finaldf[label_column], finaldf[word[0]])])
    miScoredf = pandas.DataFrame(miScore).sort_values(1,ascending=0)
    miScoredf.columns = ['Word','MI Score']
    return miScoredf

# Noun Phrase

In [23]:
# Sample text
text = """The Buddha, the Godhead, resides quite as comfortably in the circuits of a digital
computer or the gears of a cycle transmission as he does at the top of a mountain
or in the petals of a flower. To think otherwise is to demean the Buddha...which is
to demean oneself."""

In [24]:
tagger = PerceptronTagger()
# Part of Speech Tagging
# Google: https://en.wikipedia.org/wiki/Part-of-speech_tagging
pos_tag = tagger.tag
taggedToks = pos_tag(re.findall(r'\w+', text))
taggedToks

[('The', 'DT'),
 ('Buddha', 'NNP'),
 ('the', 'DT'),
 ('Godhead', 'NNP'),
 ('resides', 'VBZ'),
 ('quite', 'RB'),
 ('as', 'IN'),
 ('comfortably', 'RB'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('circuits', 'NNS'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('digital', 'JJ'),
 ('computer', 'NN'),
 ('or', 'CC'),
 ('the', 'DT'),
 ('gears', 'NNS'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('cycle', 'NN'),
 ('transmission', 'NN'),
 ('as', 'IN'),
 ('he', 'PRP'),
 ('does', 'VBZ'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('top', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('mountain', 'NN'),
 ('or', 'CC'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('petals', 'NNS'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('flower', 'NN'),
 ('To', 'TO'),
 ('think', 'VB'),
 ('otherwise', 'RB'),
 ('is', 'VBZ'),
 ('to', 'TO'),
 ('demean', 'VB'),
 ('the', 'DT'),
 ('Buddha', 'NNP'),
 ('which', 'WDT'),
 ('is', 'VBZ'),
 ('to', 'TO'),
 ('demean', 'VB'),
 ('oneself', 'PRP')]

In [26]:
# This grammar is described in the paper by S. N. Kim,
# T. Baldwin, and M.-Y. Kan.
# Evaluating n-gram based evaluation metrics for automatic
# keyphrase extraction.
# Technical report, University of Melbourne, Melbourne 2010.
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""

In [27]:
# Create phrase tree
chunker = nltk.RegexpParser(grammar)
tree= chunker.parse(taggedToks)

In [28]:
# Noun Phrase Extraction Support Functions
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

# generator, generate leaves one by one
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP' or t.label()=='JJ' or t.label()=='RB'):
        yield subtree.leaves()

# stemming, lematizing, lower case... 
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word)
    return word

# stop-words and length control
def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted

# generator, create item once a time
def get_terms(tree):
    for leaf in leaves(tree):
        term = [normalise(w) for w,t in leaf if acceptable_word(w) ]
        # Phrase only
        if len(term)>1:
            yield term

In [29]:
# Traverse tree and get noun phrases
npTokenList = [word for word in get_terms(tree)]

npTokenList

[['digit', 'comput'], ['cycl', 'transmiss']]

In [30]:
# Flatten phrase lists to get tokens for analysis
def flatten(npTokenList):
    finalList =[]
    for phrase in npTokenList:
        token = ''
        for word in phrase:
            token += word + ' '
        finalList.append(token.rstrip())
    return finalList

In [31]:
finalList = flatten(npTokenList)

In [32]:
finalList

['digit comput', 'cycl transmiss']

In [57]:
from ast import literal_eval

# Revise the previous dataframe transform function...
def newDataFrameTransformation(hotelDf, reviewDF, reviewcol_name = 'reviewCol',k= 1000):
    reviews = reviewDF[reviewcol_name].values
    
    
    # Top-k frequent terms
    counter = Counter()
    for review in tqdm(reviews):
            counter.update(flatten([word
                            for word 
                            in get_terms(chunker.parse(pos_tag(re.findall(r'\w+', review)))) 
                            ]))
    topk = counter.most_common(k)        
    
    #Find out if a particular review has the word from topk list
    freqReview = []
    for i in tqdm(range(len(reviews))):
        tempCounter = Counter(flatten([word 
                                       for word 
                                       in get_terms(chunker.parse(pos_tag(re.findall(r'\w+',reviews[i]))))]))
        topkinReview = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
        freqReview.append(topkinReview)
         
    #Prepare freqReviewDf
    freqReviewDf = pandas.DataFrame(freqReview)
    dfName = []
    for c in topk:
        dfName.append(c[0])
    freqReviewDf.columns = dfName
    finalreviewDf = reviewDF.join(freqReviewDf)
    finaldf = hotelDf[['business_id','rating','Binary','review','conca_review','UserIndex','ItemIndex']].join(finalreviewDf)
    return topk, finaldf

topk_phrase, finaldf_phrase = newDataFrameTransformation(cur_df, reviewDF, k = 1000)

topk_phrase

In [43]:
# Compute PMI for all terms and all possible labels
def pmiForAllCal(df, label_column='Binary', topk=topk_phrase):
    #Try calculate all the pmi for top k and store them into one pmidf dataframe
    pmilist = []
    pmiposlist = []
    pmineglist = []
    for word in tqdm(topk):
#         pmilist.append([word[0]]+[pmiCal(df,word[0])])
        pmiposlist.append([word[0]]+[pmiIndivCal(df,word[0],1,label_column)])
        pmineglist.append([word[0]]+[pmiIndivCal(df,word[0],0,label_column)])
    pmidf = pandas.DataFrame(pmilist)
    pmiposlist = pandas.DataFrame(pmiposlist)
    pmineglist = pandas.DataFrame(pmineglist)
    pmiposlist.columns = ['word','pmi']
    pmineglist.columns = ['word','pmi']
#     pmidf.columns = ['word','pmi']
#     return pmiposlist, pmineglist, pmidf
    return pmiposlist, pmineglist

def pmiIndivCal(df,x,gt, label_column='Binary'):
    px = sum(df[label_column]==gt)/len(df)
    py = sum(df[x]==1)/len(df)
    pxy = len(df[(df[label_column]==gt) & (df[x]==1)])/len(df)
    if pxy==0:#Log 0 cannot happen
        pmi = math.log((pxy+0.0001)/(px*py))
    else:
        pmi = math.log(pxy/(px*py))
    return pmi

# Simple example of getting pairwise mutual information of a term
def pmiCal(df, x):
    pmilist=[]
    for i in [1,0]:
        for j in [0,1]:
            px = sum(df['Binary']==i)/len(df)
            py = sum(df[x]==j)/len(df)
            pxy = len(df[(df['Binary']==i) & (df[x]==j)])/len(df)
            if pxy==0:#Log 0 cannot happen
                pmi = math.log((pxy+0.0001)/(px*py))
            else:
                pmi = math.log(pxy/(px*py))
            pmilist.append([i]+[j]+[px]+[py]+[pxy]+[pmi])
    pmidf = pandas.DataFrame(pmilist)
    pmidf.columns = ['x','y','px','py','pxy','pmi']
    return pmidf

pmiposlist, pmineglist = pmiForAllCal(finaldf_phrase, topk=topk_phrase)

pmiposlist_vader, pmineglist_vader = pmiForAllCal(finaldf_phrase, topk=topk_phrase,label_column='vader')

In [76]:
pmiposlist_vader

,word,pmi
0,first time,inf
1,ice cream,inf
2,great place,inf
3,next time,inf
4,littl bit,inf
5,long time,inf
6,last time,inf
7,good thing,inf
8,good place,inf
9,dim sum,inf


In [62]:
import pickle as pkl
with open("pmineglist", "wb") as output_file:
    pkl.dump(pmineglist, output_file)
with open("pmiposlist", "wb") as output_file:
    pkl.dump(pmiposlist, output_file)

In [69]:
# topk_phrase, finaldf_phrase
with open("finaldf_phrase", "wb") as output_file:
    pkl.dump(finaldf_phrase, output_file)


## Keyphrase for positive and negative rating

In [87]:
neg_rating = pmineglist.sort_values('pmi',ascending=0)
neg_rating = neg_rating['word'][:50].tolist()

In [91]:
neg_rating

['bad place',
 'okay noth',
 'decent place',
 'ok noth',
 'decent food',
 'second chanc',
 'terribl servic',
 'mediocr food',
 'decent servic',
 'eye contact',
 'sub par',
 'slow servic',
 'high hope',
 'dri side',
 'bit bland',
 'separ bill',
 'high price',
 'empti tabl',
 'poor servic',
 'room temperatur',
 'littl bland',
 'good dish',
 'bad tast',
 'averag price',
 'asian legend',
 'quick meal',
 'good overal',
 'bad servic',
 'salti side',
 'high side',
 'swiss chalet',
 'non exist',
 'plu side',
 'extra star',
 'wow factor',
 'long wait time',
 'bad day',
 'dim sum place',
 'bit pricey',
 'instant noodl',
 'chicken piec',
 'good locat',
 'small portion',
 'beef noodl',
 'good place',
 'much sauc',
 'decent portion',
 'good noth',
 'deer garden',
 'la carnita']

In [88]:
print ('[%s]' % ', '.join(map(str, neg_rating)))

[bad place, okay noth, decent place, ok noth, decent food, second chanc, terribl servic, mediocr food, decent servic, eye contact, sub par, slow servic, high hope, dri side, bit bland, separ bill, high price, empti tabl, poor servic, room temperatur, littl bland, good dish, bad tast, averag price, asian legend, quick meal, good overal, bad servic, salti side, high side, swiss chalet, non exist, plu side, extra star, wow factor, long wait time, bad day, dim sum place, bit pricey, instant noodl, chicken piec, good locat, small portion, beef noodl, good place, much sauc, decent portion, good noth, deer garden, la carnita]


In [89]:
pos_rating = pmiposlist.sort_values('pmi',ascending=0)
pos_rating = pos_rating['word'][:50].tolist()

In [90]:
print ('[%s]' % ', '.join(map(str, pos_rating)))

[amaz experi, favourit restaur, amaz servic, wonder experi, amaz food, great recommend, favorit place, hidden gem, hair cut, perfect balanc, favourit place, littl gem, much fun, favourit spot, escap room, super help, great coffe, real deal, great custom servic, person favourit, great qualiti, top notch, tast menu, delici food, great job, super nice, awesom place, great experi, delici meal, excel servic, perfect amount, singl time, mani flavour, great staff, nail salon, farmer market, bang bang, sure everyth, perfect place, sushi bar, favourit thing, great servic, vietnames coffe, special occas, great price, good reason, foie gra, friendli staff, board game, filet mignon]


In [92]:
pos_rating

['amaz experi',
 'favourit restaur',
 'amaz servic',
 'wonder experi',
 'amaz food',
 'great recommend',
 'favorit place',
 'hidden gem',
 'hair cut',
 'perfect balanc',
 'favourit place',
 'littl gem',
 'much fun',
 'favourit spot',
 'escap room',
 'super help',
 'great coffe',
 'real deal',
 'great custom servic',
 'person favourit',
 'great qualiti',
 'top notch',
 'tast menu',
 'delici food',
 'great job',
 'super nice',
 'awesom place',
 'great experi',
 'delici meal',
 'excel servic',
 'perfect amount',
 'singl time',
 'mani flavour',
 'great staff',
 'nail salon',
 'farmer market',
 'bang bang',
 'sure everyth',
 'perfect place',
 'sushi bar',
 'favourit thing',
 'great servic',
 'vietnames coffe',
 'special occas',
 'great price',
 'good reason',
 'foie gra',
 'friendli staff',
 'board game',
 'filet mignon']

# Vader 

In [100]:
# Get vader binary values
finaldf_phrase['Binary_vader'] = (finaldf_phrase['vader'] > 0.05)*[1]

In [102]:
pmiposlist_vader, pmineglist_vader = pmiForAllCal(finaldf_phrase, topk=topk_phrase,label_column='Binary_vader')



  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

  0%|                                                                                 | 1/1000 [00:00<15:14,  1.09it/s]

  0%|▏                                                                                | 2/1000 [00:01<14:45,  1.13it/s]

  0%|▏                                                                                | 3/1000 [00:02<14:20,  1.16it/s]

  0%|▎                                                                                | 4/1000 [00:03<14:02,  1.18it/s]

  0%|▍                                                                                | 5/1000 [00:04<13:48,  1.20it/s]

  1%|▍                                                                                | 6/1000 [00:04<13:37,  1.22it/s]

  1%|▌                                                                                | 7/1000 [00:05<13:29,  1.23it/s]

  1%|▋                        

  7%|█████▎                                                                          | 67/1000 [00:53<12:22,  1.26it/s]

  7%|█████▍                                                                          | 68/1000 [00:54<12:20,  1.26it/s]

  7%|█████▌                                                                          | 69/1000 [00:54<12:17,  1.26it/s]

  7%|█████▌                                                                          | 70/1000 [00:55<12:18,  1.26it/s]

  7%|█████▋                                                                          | 71/1000 [00:56<12:15,  1.26it/s]

  7%|█████▊                                                                          | 72/1000 [00:57<12:06,  1.28it/s]

  7%|█████▊                                                                          | 73/1000 [00:57<11:58,  1.29it/s]

  7%|█████▉                                                                          | 74/1000 [00:58<12:00,  1.29it/s]

  8%|██████                     

 13%|██████████▌                                                                    | 134/1000 [01:45<10:59,  1.31it/s]

 14%|██████████▋                                                                    | 135/1000 [01:46<10:56,  1.32it/s]

 14%|██████████▋                                                                    | 136/1000 [01:47<10:55,  1.32it/s]

 14%|██████████▊                                                                    | 137/1000 [01:48<10:55,  1.32it/s]

 14%|██████████▉                                                                    | 138/1000 [01:48<10:53,  1.32it/s]

 14%|██████████▉                                                                    | 139/1000 [01:49<10:53,  1.32it/s]

 14%|███████████                                                                    | 140/1000 [01:50<10:54,  1.31it/s]

 14%|███████████▏                                                                   | 141/1000 [01:51<10:56,  1.31it/s]

 14%|███████████▏               

 20%|███████████████▉                                                               | 201/1000 [02:36<10:03,  1.32it/s]

 20%|███████████████▉                                                               | 202/1000 [02:37<10:03,  1.32it/s]

 20%|████████████████                                                               | 203/1000 [02:38<10:02,  1.32it/s]

 20%|████████████████                                                               | 204/1000 [02:39<10:01,  1.32it/s]

 20%|████████████████▏                                                              | 205/1000 [02:39<10:04,  1.32it/s]

 21%|████████████████▎                                                              | 206/1000 [02:40<10:06,  1.31it/s]

 21%|████████████████▎                                                              | 207/1000 [02:41<10:05,  1.31it/s]

 21%|████████████████▍                                                              | 208/1000 [02:42<10:05,  1.31it/s]

 21%|████████████████▌          

 27%|█████████████████████▏                                                         | 268/1000 [03:27<09:15,  1.32it/s]

 27%|█████████████████████▎                                                         | 269/1000 [03:28<09:12,  1.32it/s]

 27%|█████████████████████▎                                                         | 270/1000 [03:29<09:12,  1.32it/s]

 27%|█████████████████████▍                                                         | 271/1000 [03:30<09:11,  1.32it/s]

 27%|█████████████████████▍                                                         | 272/1000 [03:30<09:09,  1.32it/s]

 27%|█████████████████████▌                                                         | 273/1000 [03:31<09:09,  1.32it/s]

 27%|█████████████████████▋                                                         | 274/1000 [03:32<09:08,  1.32it/s]

 28%|█████████████████████▋                                                         | 275/1000 [03:33<09:07,  1.32it/s]

 28%|█████████████████████▊     

...

 67%|████████████████████████████████████████████████████▉                          | 670/1000 [08:32<04:10,  1.32it/s]

 67%|█████████████████████████████████████████████████████                          | 671/1000 [08:32<04:08,  1.32it/s]

 67%|█████████████████████████████████████████████████████                          | 672/1000 [08:33<04:07,  1.32it/s]

 67%|█████████████████████████████████████████████████████▏                         | 673/1000 [08:34<04:06,  1.33it/s]

 67%|█████████████████████████████████████████████████████▏                         | 674/1000 [08:35<04:05,  1.33it/s]

 68%|█████████████████████████████████████████████████████▎                         | 675/1000 [08:35<04:04,  1.33it/s]

 68%|█████████████████████████████████████████████████████▍                         | 676/1000 [08:36<04:03,  1.33it/s]

 68%|█████████████████████████████████████████████████████▍                         | 677/1000 [08:37<04:03,  1.33it/s]

 68%|███████████████████████████

 74%|██████████████████████████████████████████████████████████▏                    | 737/1000 [09:22<03:19,  1.32it/s]

 74%|██████████████████████████████████████████████████████████▎                    | 738/1000 [09:23<03:18,  1.32it/s]

 74%|██████████████████████████████████████████████████████████▍                    | 739/1000 [09:24<03:17,  1.32it/s]

 74%|██████████████████████████████████████████████████████████▍                    | 740/1000 [09:25<03:16,  1.32it/s]

 74%|██████████████████████████████████████████████████████████▌                    | 741/1000 [09:25<03:15,  1.33it/s]

 74%|██████████████████████████████████████████████████████████▌                    | 742/1000 [09:26<03:13,  1.33it/s]

 74%|██████████████████████████████████████████████████████████▋                    | 743/1000 [09:27<03:13,  1.33it/s]

 74%|██████████████████████████████████████████████████████████▊                    | 744/1000 [09:28<03:12,  1.33it/s]

 74%|███████████████████████████

 80%|███████████████████████████████████████████████████████████████▌               | 804/1000 [10:13<02:27,  1.33it/s]

 80%|███████████████████████████████████████████████████████████████▌               | 805/1000 [10:14<02:26,  1.33it/s]

 81%|███████████████████████████████████████████████████████████████▋               | 806/1000 [10:15<02:25,  1.33it/s]

 81%|███████████████████████████████████████████████████████████████▊               | 807/1000 [10:15<02:25,  1.33it/s]

 81%|███████████████████████████████████████████████████████████████▊               | 808/1000 [10:16<02:24,  1.33it/s]

 81%|███████████████████████████████████████████████████████████████▉               | 809/1000 [10:17<02:23,  1.33it/s]

 81%|███████████████████████████████████████████████████████████████▉               | 810/1000 [10:18<02:22,  1.33it/s]

 81%|████████████████████████████████████████████████████████████████               | 811/1000 [10:18<02:22,  1.33it/s]

 81%|███████████████████████████

 87%|████████████████████████████████████████████████████████████████████▊          | 871/1000 [11:04<01:37,  1.32it/s]

 87%|████████████████████████████████████████████████████████████████████▉          | 872/1000 [11:05<01:36,  1.33it/s]

 87%|████████████████████████████████████████████████████████████████████▉          | 873/1000 [11:05<01:35,  1.33it/s]

 87%|█████████████████████████████████████████████████████████████████████          | 874/1000 [11:06<01:34,  1.33it/s]

 88%|█████████████████████████████████████████████████████████████████████▏         | 875/1000 [11:07<01:33,  1.33it/s]

 88%|█████████████████████████████████████████████████████████████████████▏         | 876/1000 [11:08<01:33,  1.33it/s]

 88%|█████████████████████████████████████████████████████████████████████▎         | 877/1000 [11:08<01:32,  1.33it/s]

 88%|█████████████████████████████████████████████████████████████████████▎         | 878/1000 [11:09<01:31,  1.33it/s]

 88%|███████████████████████████

 94%|██████████████████████████████████████████████████████████████████████████     | 938/1000 [11:55<00:46,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▏    | 939/1000 [11:55<00:46,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▎    | 940/1000 [11:56<00:45,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▎    | 941/1000 [11:57<00:44,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▍    | 942/1000 [11:58<00:43,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▍    | 943/1000 [11:58<00:42,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▌    | 944/1000 [11:59<00:42,  1.33it/s]

 94%|██████████████████████████████████████████████████████████████████████████▋    | 945/1000 [12:00<00:41,  1.33it/s]

 95%|███████████████████████████

In [103]:
pos_rating = pmiposlist_vader.sort_values('pmi',ascending=0)
pos_rating = pos_rating['word'][:50].tolist()

In [104]:
pos_rating

['nice experi',
 'dim light',
 'great atmospher',
 'good care',
 'special event',
 'nice balanc',
 'super nice',
 'nice crunch',
 'great recommend',
 'perfect size',
 'nice chang',
 'great addit',
 'perfect amount',
 'perfect balanc',
 'gener amount',
 'interior design',
 'super tender',
 'great spot',
 'great textur',
 'great job',
 'great option',
 'amaz food',
 'good vibe',
 'welcom addit',
 'sweet tooth',
 'littl gem',
 'nice surpris',
 'winterlici menu',
 'nice textur',
 'great vibe',
 'great choic',
 'super help',
 'littl salti',
 'delici servic',
 'perfect spot',
 'great coffe',
 'larg window',
 'nice kick',
 'favourit part',
 'grill octopu',
 'special occas',
 'brussel sprout',
 'good friend',
 'nice place',
 'duck confit',
 'much fun',
 'great way',
 'summer time',
 'pleasant surpris',
 'good balanc']

In [106]:
neg_rating = pmineglist_vader.sort_values('pmi',ascending=0)
neg_rating = neg_rating['word'][:50].tolist()

In [107]:
print ('[%s]' % ', '.join(map(str, pos_rating)))

[nice experi, dim light, great atmospher, good care, special event, nice balanc, super nice, nice crunch, great recommend, perfect size, nice chang, great addit, perfect amount, perfect balanc, gener amount, interior design, super tender, great spot, great textur, great job, great option, amaz food, good vibe, welcom addit, sweet tooth, littl gem, nice surpris, winterlici menu, nice textur, great vibe, great choic, super help, littl salti, delici servic, perfect spot, great coffe, larg window, nice kick, favourit part, grill octopu, special occas, brussel sprout, good friend, nice place, duck confit, much fun, great way, summer time, pleasant surpris, good balanc]


In [108]:
print ('[%s]' % ', '.join(map(str, neg_rating)))

[terribl servic, poor servic, bad tast, bad servic, eye contact, bad day, mediocr food, bad review, bad experi, empti tabl, sub par, canadian tire, tini piec, food order, half way, custom servic, stinki tofu, neg review, instant noodl, room temperatur, cash regist, full price, first place, entir restaur, multipl time, ten minut, numer time, nail salon, last time, half hour, dim light, high price, drink order, high hope, everi time, pita bread, front desk, great recommend, nice balanc, special event, entir time, whole time, phone number, last visit, small piec, asian legend, kitchen staff, hair cut, posit review, star review]
